In [12]:
!pip install nltk

In [43]:
!pip install keras

In [76]:
import nltk
import numpy as np
from nltk.corpus import brown
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

### **Préparation des données**

In [77]:
#Téléchargement du corpus Brown
nltk.download('brown')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [78]:
#Prétraitement des données
sentences=brown.tagged_sents(tagset='universal')  #Récupère les phrases annotées du corpus brown avec le jeu de tags universel.
sentences

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

In [79]:
#Création d'un vocabulaire
words = set(word.lower() for sentence in sentences for word, tag in sentence)
word_to_index ={word: i+1 for i, word in enumerate(words)}
tag_to_index ={tag: i for i, tag in enumerate(set(tag for sentence in sentences for _, tag in sentence))}
len(words)

49815

In [80]:
#Préparation des données d'entraînement
X = [[word_to_index.get(word.lower(), 0) for word,_ in sentence] for sentence in sentences] #cherche word in sentence et affiche l'indice de ce word s'il le trouve sinon donne 0
y = [[tag_to_index[tag] for _, tag in sentence] for sentence in sentences]
X_padded = pad_sequences(X, padding='post') # pour que toutes les séquences aient la même longueur en ajoutant des zéros à la fin des séquences plus courtes.
y_padded = pad_sequences(y, padding='post')
y_padded=[to_categorical(i, num_classes=len(tag_to_index)) for i in y_padded] #convertit chaque séquence de tags en une séquence de vecteurs one-hot encodés
len(X)


57340

In [81]:
# Split des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_padded, np.array(y_padded), test_size=0.2, random_state=42)

### **Construction du modèle**

In [82]:
!pip install tensorflow

In [83]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense

In [84]:
# Créer un modèle séquentiel
model = Sequential()
#Création du modèle RNN pour l'étiquetage POS model Sequential ()
model.add(Embedding(input_dim=len(words)+1, output_dim=128, input_length=X_padded.shape[1]))
model.add(SimpleRNN(units=128, return_sequences=True))  ## Couche RNN avec 128 unités
model.add(Dense (units=len (tag_to_index), activation='softmax')) #couche de sortie
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #perte utilisée est l'entropie croisée catégorielle, adaptée pour les tâches de classification multiclasse.

In [85]:
#Entraînement du modèle
model.fit(X_train, np.array(y_train), batch_size=32, epochs=5, validation_split=0.1)


Epoch 1/5
1291/1291 [==============================] - 143s 110ms/step - loss: 0.0773 - accuracy: 0.9793 - val_loss: 0.0167 - val_accuracy: 0.9944
Epoch 2/5
1291/1291 [==============================] - 128s 99ms/step - loss: 0.0109 - accuracy: 0.9962 - val_loss: 0.0142 - val_accuracy: 0.9949
Epoch 3/5
1291/1291 [==============================] - 126s 97ms/step - loss: 0.0074 - accuracy: 0.9973 - val_loss: 0.0150 - val_accuracy: 0.9948
Epoch 4/5
1291/1291 [==============================] - 125s 97ms/step - loss: 0.0055 - accuracy: 0.9981 - val_loss: 0.0161 - val_accuracy: 0.9948
Epoch 5/5
1291/1291 [==============================] - 124s 96ms/step - loss: 0.0041 - accuracy: 0.9986 - val_loss: 0.0183 - val_accuracy: 0.9943


In [89]:
#Evaluation du modèle
loss, accuracy=model.evaluate(X_test, np.array(y_test))
print (f"Loss: {loss}, Accuracy: {accuracy}")

359/359 [==============================] - 6s 17ms/step - loss: 0.0174 - accuracy: 0.9947
Loss: 0.01736871898174286, Accuracy: 0.9947224855422974
